In [ ]:
# import package
from preprocessing import *
from features import *
from plots import *
from events import *

---------- Preprocessing ----------

In [ ]:
"""
Import your data

Function Arguments:
path     - the path to a directory, .zip, or .csv that you want to import (mandatory argument)
glucose  - name of the glucose value column in the .csv files being imported ("Glucose Value (mg/dL)" by default)
time     - name of the timestamp column in the .csv files being imported ("Timestamp (YYYY-MM-DDThh:mm:ss)" by default)
interval - resampling interval in minutes that the data should follow (5 by default)
max_gap  - the maximum amount of minutes that a gap can be before it cannot be interpolated (45 by default)
"""
df = import_data("datasets") # datasets is a directory containing 3 Dexcom-formatted .csv files
df # Pandas DataFrame with 4 columns, multi-indexed by 'id'

---------- Features ----------

In [ ]:
"""
Calculate all available metrics
"""
metrics = create_features(df)
metrics # Pandas DataFrame multi-indexed by 'id': one row per patient (.csv file) and one column per metric/statistic

In [ ]:
"""
Calculate only one metric for a single patient
"""
mage = MAGE(df.loc["ElizaSutherland"]) # using MAGE as an example
mage

---------- Event Framework ---------- \
'Events' are broadly defined as any intervals of time that are believed to be of more significance.
The package provides functionality for calculating 'episodes' (literature-defined ranges for hypo- and hyperglycemia) and 'excursions' 
(intervals where a patient's blood glucose values reach thresholds that are typically unusual for this patient), and we are currently 
working on meal detection. Alternatively, users can add in their own custom events. \
\
Each 'event' within the framework shares the same structure: the id of the patient it relates to, a timestamp indicating when the event occurs, the minutes before and after the given timestamp to also look at, the type of event it is (up to the user), and a description indicating any other relevant context about the event.

In [ ]:
"""
Retrieve the curated events that the package provides functionality for (excursions and episodes).
"""
curated_events = get_curated_events(df)
curated_events

In [ ]:
"""
Retrieve solely episodes or excursions instead
"""
episodes = get_episodes(df)
excursions = get_excursions(df)

In [ ]:
"""
Manually create custom events
"""

# created event
event = pd.DataFrame.from_records([{'id': "ElizaSutherland", 
                                    "Timestamp (YYYY-MM-DDThh:mm:ss)": '2023-07-11 12:55:32', 
                                    'Before': 10, 'After': 10, 'Type': "example event", 'Description': 'example event #1'}])

event # example of how the curated events have the same structure as the manually added one

In [ ]:
"""
Alternatively, import events from .csv files (each row within a .csv file will be imported as an event)

Function Arguments:
path     - path of a directory, .zip, or .csv to import events from
id       - id of the patient that the imported events belong to
day_col  - name of the column indicating the day (year, month, and day) that the event occurred ("Day" by default)
time_col - name of the column indicating what time during the day the event occurred ("Time" by default)
before   - minutes before the given timestamp to look at (60 by default)
after    - minutes after the given timestamp to look at (60 by default)
type     - what 'type' the imported events should be classified as ("imported event" by default)
"""
imported_events = import_events("imported_events/FakeMeals.csv", "ElizaSutherland", type="Cronometer Meals")
imported_events

In [ ]:
"""
Combining all the events together
(Also an example of how the curated events have the same structure as the imported ones)
"""
events = pd.concat([event, imported_events, curated_events]).reset_index()
events

In [ ]:
"""
Quick way to see whether your event 'types' have been consistently named
"""
event_summary(events)

In [ ]:
"""
Calculating statistics for events
"""
event_statistics(df, events) # same structure as create_features() above: multi-indexed Pandas DataFrame with one row per patient and one column per statistic

---------- Plotting ---------- \
The package provides functionality for five different plots: Daily (Time-Series), Weekly (Time-Series), Event, Spaghetti, and AGP. \
For each of these plot types, two functions are available: one that sequentially generates the respective plot for all patients/events passed, and another that only generates a plot for a specific patient/event.

In [ ]:
"""
Daily (Time-Series) Plots

Extra Function Arguments:
events - Pandas DataFrame containing events (in the same structure as in the framework above), results in vertical lines marking where the events occur on the plot (None by default)
height - height (in pixels) of the resulting plot (2000 by default)
"""
daily_plot_all(df) # plots for all patients
daily_plot(df, "NathanielBarrow", events=events) # would plot only for "NathanielBarrow" and show relevant events

In [ ]:
"""
Weekly (Time-Series) Plots

Extra Function Arguments:
height    - height (in pixels) of the resulting plot (1000 by default)
"""
weekly_plot_all(df) # plots for all patients
weekly_plot(df, "ElizaSutherland") # would plot only for "ElizaSutherland"

In [ ]:
"""
Spaghetti Plots

Extra Function Arguments:
chunk_day - boolean indicating whether to display two separate subplots based on whether values occur on a weekend or weekday (False by default)
height    - height (in pixels) of the resulting plot (600 by default)
"""
spaghetti_plot_all(df) # plots for all patients
spaghetti_plot(df, "ElizaSutherland") # would plot only for "ElizaSutherland"

In [ ]:
"""
AGP Plots

Extra Function Arguments:
height    - height (in pixels) of the resulting plot (600 by default)
"""
AGP_plot_all(df) # plots for all patients
AGP_plot(df, "ElizaSutherland") # would plot only for "PenelopeFitzroy"